# ControlNet Inpainting - Precise Edge Control

Better edge preservation than LaMa, with modern diffusion models.

In [ ]:
!pip install diffusers transformers accelerate controlnet-aux

In [ ]:
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel
import torch
from PIL import Image
import numpy as np

# Load ControlNet for inpainting
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_inpaint",
    torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# Upload and process
from google.colab import files
import zipfile

uploaded = files.upload()
zip_name = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall('.')

image = Image.open("image.png").convert("RGB")
mask = Image.open("mask.png").convert("L")

# Create control image (masked area)
control_image = Image.new('RGB', image.size, (0, 0, 0))
control_image.paste(image, mask=Image.fromarray(255 - np.array(mask)))

In [ ]:
# Inpaint with ControlNet
result = pipe(
    prompt="empty room interior, clean walls, professional photo",
    negative_prompt="furniture, objects, artifacts",
    image=image,
    mask_image=mask,
    control_image=control_image,
    num_inference_steps=30,
    guidance_scale=7.5,
    controlnet_conditioning_scale=1.0
).images[0]

result.save("result_controlnet.png")
files.download("result_controlnet.png")